## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan, when, count, col, mean

Импортирование нужных библиотек и модулей

# Подготовка данных

In [2]:
spark = (SparkSession.builder
                     .master('local')
                     .appName('Predict price project')
                     .getOrCreate())

In [3]:
RANDOM_SEED = 0

In [4]:
data = spark.read.load('/datasets/housing.csv', format='csv', inferSchema=True, header=True)

В этой части кода инициализация локальной спарк сессии, создание константы для обучения модели и чтение файла с помощью спарк


In [5]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [6]:
data.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.0035317

In [7]:
data.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


    Из printSchema можно увидеть типы данных в датасете:
        longitude — широта, float
        latitude — долгота, float
        housing_median_age — медианный возраст жителей жилого массива, float
        total_rooms — общее количество комнат в домах жилого массива, float
        total_bedrooms — общее количество спален в домах жилого массива, float
        population — количество человек, которые проживают в жилом массиве, float
        households — количество домовладений в жилом массиве, float
        median_income — медианный доход жителей жилого массива, float
        median_house_value — медианная стоимость дома в жилом массиве, float
        ocean_proximity — близость к океану, object

In [8]:
data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,0,0,0,0,0,0,0,0,0


In [9]:
data = data.na.fill({
    'total_bedrooms': data.select(mean('total_bedrooms')).first()[0]
})

Пропуски есть только в одном столбце - total_bedrooms, решил их просто удалить, т.к. они не повлияют

In [10]:
cat_cols = ['ocean_proximity']
num_cols = ['longitude', 
            'latitude', 
            'housing_median_age', 
            'total_rooms', 
            'total_bedrooms', 
            'population', 
            'households', 
            'median_income']
target = 'median_house_value'

In [11]:
train_data, test_data = data.randomSplit([.8, .2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16468 4172


In [12]:
indexer = StringIndexer(inputCols=cat_cols, 
                        outputCols=[c+'_idx' for c in cat_cols]) 
model_ind = indexer.fit(train_data)
train_data = model_ind.transform(train_data)
test_data = model_ind.transform(test_data)
cols = [c for c in train_data.columns for i in cat_cols if (c.startswith(i))]

In [13]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in cat_cols],
                        outputCols=[c+'_ohe' for c in cat_cols])
encoder_model = encoder.fit(train_data)
train_data = encoder_model.transform(train_data)
test_data = encoder_model.transform(test_data)
cols = [c for c in train_data.columns for i in cat_cols if (c.startswith(i))]

In [14]:
categorical_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in cat_cols],
                                        outputCol='categorical_features')
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

In [15]:
numerical_assembler = VectorAssembler(inputCols=num_cols, outputCol='numerical_features')
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

In [16]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
scaler = standardScaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [17]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol='features') 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

Преобразование категориального столбца OneHotEncoder'ом и его векторизация

Преобразование числовых столбцов StandartScaler'ом и их векторизация

In [18]:
train_data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|numerical_features_scaled|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|[

Видно, что появились колонки numerical_features_scaled и categorical_features, они будут использоваться для обучения модели

Разделение на тренировочную и тестовую выборки с размерами 80 на 20 соответственно

# Обучение моделей

In [19]:
lr = LinearRegression(labelCol=target, featuresCol='features', regParam=0.1)
first_model = lr.fit(train_data)

24/11/24 19:23:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/11/24 19:23:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/11/24 19:23:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/11/24 19:23:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


Создание и обучение модели линейной регрессии

In [20]:
predictions_1 = first_model.transform(test_data)

predictedLabes = predictions_1.select('median_house_value', 'prediction')
predictedLabes.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           70200.0|167962.64249178115|
|           70500.0|142686.32015178865|
|           68300.0| 143312.8483612081|
|           70500.0|162477.70521104848|
|           60000.0|141126.10796662746|
|           85600.0|187015.75707984623|
|           82000.0|199150.46138153225|
|           85400.0|189559.56055552838|
|           75000.0|103783.36906835949|
|          104200.0|198148.06605672697|
|           92500.0|166343.47047081823|
|           76800.0|146931.20835949527|
|           66800.0|132545.52288115025|
|          100500.0| 164427.1014370937|
|           94800.0| 228655.3594511128|
|           83000.0|176455.80387839815|
|           81100.0| 149150.1242510574|
|           78400.0|121413.13126133056|
|           97900.0|134889.05861486867|
|          135600.0| 171866.1898276303|
+------------------+------------------+
only showing top 20 rows



In [21]:
metrics = ['rmse', 'mae', 'r2']
evaluators = [RegressionEvaluator(labelCol='median_house_value', 
                                  predictionCol='prediction',
                                  metricName=metric) for metric in metrics]

for evaluator, metric in zip(evaluators, metrics):
    value = evaluator.evaluate(predictedLabes)
    print(f'{metric.upper()}: {round(value, 3)}')

RMSE: 68487.642
MAE: 49735.665
R2: 0.658


Из метрик на тестовой выборке, можно сказать, что результаты не очень хорошие, нужна работа с фичами

# Обучение второй модели

In [22]:
num_cols = ['longitude', 
            'latitude',
            'housing_median_age', 
            'total_rooms', 
            'total_bedrooms', 
            'population', 
            'households', 
            'median_income']
target = 'median_house_value'

In [23]:
lr = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', regParam=0.1)
second_model = lr.fit(train_data)

Создание и обучение модели линейной регрессии, но без категориального столбца

In [24]:
predictions_2 = second_model.transform(test_data)

predictedLabes = predictions_2.select('median_house_value', 'prediction')
predictedLabes.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           70200.0|135889.58975230576|
|           70500.0|109640.21101303352|
|           68300.0| 110460.9615805326|
|           70500.0|130276.26485034358|
|           60000.0|110098.97620087722|
|           85600.0| 155122.4206099878|
|           82000.0|167694.93087517936|
|           85400.0|157771.73178562569|
|           75000.0| 49695.49364908319|
|          104200.0|165918.87582880864|
|           92500.0|139951.77544687595|
|           76800.0|115670.16127524106|
|           66800.0|101687.71240268275|
|          100500.0|131343.79377482552|
|           94800.0|208840.90257349703|
|           83000.0|142721.74608319625|
|           81100.0|113257.37599718664|
|           78400.0| 83306.75305129075|
|           97900.0| 96144.53529526945|
|          135600.0|149099.04425167106|
+------------------+------------------+
only showing top 20 rows



In [25]:
metrics = ['rmse', 'mae', 'r2']
evaluators = [RegressionEvaluator(labelCol='median_house_value', 
                                  predictionCol='prediction',
                                  metricName=metric) for metric in metrics]

for evaluator, metric in zip(evaluators, metrics):
    value = evaluator.evaluate(predictedLabes)
    print(f'{metric.upper()}: {round(value, 3)}')

RMSE: 69374.078
MAE: 50761.896
R2: 0.649


Результаты стали еще хуже, удаление категориального признака точность у модели не прибавила

In [26]:
spark.stop()

# Анализ результатов

    Метрики первой модели, в которую включены все столбцы:
        RMSE: 66186.54
        MAE: 48599.695
        R2: 0.667

    Метрики второй модели, из которой исключен категориальный столбец:
        RMSE: 66920.053                                                                    
        MAE: 49578.919
        R2: 0.66

В целом, у обоих моделей метрики не очень хорошие, RMSE очень большой, если вопрос, что нужно ли оставить категориальный столбец, то ответ будет, что нужно и влияние он имеет, хоть и небольшое, чтобы повысить метрики, мне кажется, нужна работа с признаками, создание новых и работа с моделью, то есть, перебор гипермараметров, также можно использовать другую регрессионую модель